# <font color='cyan'>Chatbot API with Keras Model</font>

### Main challenges:
- Classify user input to recognise intent
- keep context

In [1]:
# Keras deep learning library to build classification model
# Lancaster stemming library used to collapse distinct word forms
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

Using TensorFlow backend.


In [3]:
'''
Chatbot intents & patterns to learn defined in a plain JSON file
Classification model can be created for small vocabulary
Need to build vocabulary > patterns processed
each word stemmed to produce generic root > help cover more combi for inputs
'''

'\nChatbot intents & patterns to learn defined in a plain JSON file\nClassification model can be created for small vocabulary\nNeed to build vocabulary > patterns processed\neach word stemmed to produce generic root > help cover more combi for inputs\n'

### Read the JSON file

In [4]:
import json

with open('chat_intents.json') as intent_file:
    intents = json.load(intent_file)

In [5]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/asienwald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']

#loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in sentence
        w = nltk.word_tokenize(pattern)
        
        # add to words list
        words.extend(w)
        
        # add documents in corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem & lower ea word & remove dupes
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns & intents
print(len(documents), 'documents', documents)
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocab
print(len(words), "unique stemmed words", words)

58 documents [(['are', 'cca', 'compulsory'], 'cca_compulsory'), (['do', 'i', 'need', 'to', 'join', 'a', 'cca'], 'cca_compulsory'), (['can', 'I', 'not', 'join', 'a', 'club'], 'cca_compulsory'), (['do', 'i', 'have', 'to', 'join', 'a', 'cca'], 'cca_compulsory'), (['how', 'do', 'i', 'apply', 'for', 'a', 'cca'], 'cca_application'), (['how', 'to', 'join', 'cca'], 'cca_application'), (['register', 'for', 'a', 'cca'], 'cca_application'), (['quit', 'cca'], 'cca_quit'), (['leave', 'my', 'club', 'or', 'cca'], 'cca_quit'), (['retire', 'from', 'my', 'cca'], 'cca_quit'), (['how', 'to', 'create', 'cca'], 'cca_creation'), (['make', 'new', 'cca'], 'cca_creation'), (['start', 'a', 'cca'], 'cca_creation'), (['register', 'for', 'a', 'new', 'cca', 'or', 'club'], 'cca_creation'), (['benefits', 'of', 'cca'], 'cca_benefits'), (['why', 'do', 'i', 'need', 'to', 'join', 'a', 'cca'], 'cca_benefits'), (['is', 'cca', 'important'], 'cca_benefits'), (['what', 'ccas', 'are', 'there'], 'cca_list'), (['what', 'cca', 'do

In [7]:
'''
Training won't be run based on vocab of words (words meaningless for machine)
Need to translate words into bags of words with arr containing 0/1

Arr length will be equal to vocab size & 1 set when word from current
pattern is located in a given position
'''

"\nTraining won't be run based on vocab of words (words meaningless for machine)\nNeed to translate words into bags of words with arr containing 0/1\n\nArr length will be equal to vocab size & 1 set when word from current\npattern is located in a given position\n"

In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [9]:
# training set, bag of words for ea sentence
for doc in documents:
    # initialise our bag of words
    bag = []
    #list tokenized words for the pattern
    pattern_words = doc[0]
    # stem ea word - create base word to represent related words
    # Please read stemming VS lemmatization
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words arr with 1, if word match found in
    # current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # output is '0' for ea tag & '1' for current tag (for ea pattern)
    # output_row is basically the class/intents it falls in
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
    # print(training)

In [10]:
# shuffle our features & turn into np.array
random.shuffle(training)
training = np.array(training)

In [11]:
# create train & test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
#with open('model.pickle', 'wb') as f:
#  pickle.dump((words, labels, training))

In [13]:
'''
Training data - X (pattern converted into array [0,1,0,1...,0])
Y (intents converted into arr [1,0,0,0,..,0]), will be single 1 for intents arr

Model built on 3 layers

Classification output will be multiclass arr > help identify encoded intent

Use Softmax activation to produce multiclass classification output
(result returns arr of 0/1: [1,0,0...,0] - identifies encoded intent)
'''

'\nTraining data - X (pattern converted into array [0,1,0,1...,0])\nY (intents converted into arr [1,0,0,0,..,0]), will be single 1 for intents arr\n\nModel built on 3 layers\n\nClassification output will be multiclass arr > help identify encoded intent\n\nUse Softmax activation to produce multiclass classification output\n(result returns arr of 0/1: [1,0,0...,0] - identifies encoded intent)\n'

## <font color='red'>**NOTE</font> <br>
Ok so Cael & Joe, <br>
This part below is basically what makes up a Neural network in ML

So basically a neural network has different layers in its hidden layer (main layer)
that tries to sift out features to identify what is going on and learn

In each layer we have number of neurons to help us know what's going on <br>
> Simple Eg. When you touch something you have nerves to tell you if you really touched something or not

Softmax is the activation function we're using to determine when to trigger these neurons <br>
> Eg. Did you really touch something?? How do you know? > Your nerves felt something and sent signals to your brain

Input shape = specify shape of your data its dealing with in 1st layer

In [14]:
# Create model - 3 layers
# 1st layer 128 neurons, 2nd layer 64 neurons & 3rd contains num of neurons
# equal to num of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]), ), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

### Compile Keras model with SGD optimiser

In [15]:
# Compile model. Stochastic gradient descent with Nesterov accelerated
# gradient gives good results for this model

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
# print(dir(sgd))
# print(sgd.lr)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd,
              metrics = ['accuracy'])

### Fit model

In [16]:
# Execute training & construct classification model

In [17]:
# Fit the model
# Epochs means iterations
model.fit(np.array(train_x), np.array(train_y), epochs = 200,
          batch_size = 5, verbose = 1)

Epoch 1/200
58/58 [==============================] - 0s 2ms/step - loss: 2.7052 - accuracy: 0.0690
Epoch 2/200
58/58 [==============================] - 0s 235us/step - loss: 2.6021 - accuracy: 0.1207
Epoch 3/200
58/58 [==============================] - 0s 249us/step - loss: 2.5443 - accuracy: 0.1379
Epoch 4/200
58/58 [==============================] - 0s 239us/step - loss: 2.4814 - accuracy: 0.1897
Epoch 5/200
58/58 [==============================] - 0s 248us/step - loss: 2.3792 - accuracy: 0.2586
Epoch 6/200
58/58 [==============================] - 0s 251us/step - loss: 2.3495 - accuracy: 0.2931
Epoch 7/200
58/58 [==============================] - 0s 265us/step - loss: 2.2385 - accuracy: 0.3966
Epoch 8/200
58/58 [==============================] - 0s 276us/step - loss: 2.1118 - accuracy: 0.4483
Epoch 9/200
58/58 [==============================] - 0s 246us/step - loss: 2.0498 - accuracy: 0.4138
Epoch 10/200
58/58 [==============================] - 0s 252us/step - loss: 1.8128 - accuracy

58/58 [==============================] - 0s 205us/step - loss: 0.1314 - accuracy: 0.9828
Epoch 82/200
58/58 [==============================] - 0s 210us/step - loss: 0.1358 - accuracy: 0.9310
Epoch 83/200
58/58 [==============================] - 0s 172us/step - loss: 0.1202 - accuracy: 0.9828
Epoch 84/200
58/58 [==============================] - 0s 163us/step - loss: 0.1415 - accuracy: 0.9655
Epoch 85/200
58/58 [==============================] - 0s 180us/step - loss: 0.0946 - accuracy: 0.9828
Epoch 86/200
58/58 [==============================] - 0s 184us/step - loss: 0.2463 - accuracy: 0.9138
Epoch 87/200
58/58 [==============================] - 0s 190us/step - loss: 0.1303 - accuracy: 0.9828
Epoch 88/200
58/58 [==============================] - 0s 212us/step - loss: 0.1236 - accuracy: 0.9828
Epoch 89/200
58/58 [==============================] - 0s 191us/step - loss: 0.1165 - accuracy: 0.9655
Epoch 90/200
58/58 [==============================] - 0s 174us/step - loss: 0.1693 - accuracy: 

58/58 [==============================] - 0s 174us/step - loss: 0.0399 - accuracy: 1.0000
Epoch 161/200
58/58 [==============================] - 0s 251us/step - loss: 0.0804 - accuracy: 0.9828
Epoch 162/200
58/58 [==============================] - 0s 198us/step - loss: 0.0262 - accuracy: 1.0000
Epoch 163/200
58/58 [==============================] - 0s 174us/step - loss: 0.0447 - accuracy: 0.9828
Epoch 164/200
58/58 [==============================] - 0s 198us/step - loss: 0.0644 - accuracy: 0.9828
Epoch 165/200
58/58 [==============================] - 0s 212us/step - loss: 0.1201 - accuracy: 0.9310
Epoch 166/200
58/58 [==============================] - 0s 216us/step - loss: 0.0448 - accuracy: 1.0000
Epoch 167/200
58/58 [==============================] - 0s 213us/step - loss: 0.0338 - accuracy: 0.9828
Epoch 168/200
58/58 [==============================] - 0s 190us/step - loss: 0.0577 - accuracy: 0.9828
Epoch 169/200
58/58 [==============================] - 0s 208us/step - loss: 0.0645 - a

### Define helper functions

In [18]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into arr
    sentence_words = nltk.word_tokenize(sentence)
    # stem ea word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [19]:
# Translate user sentence into bag of words with arr 0/1
# 0 or 1 for ea word in bag that exists in the sentence
def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # Initialise bag of words - matrix of N words, vocab matrix
    bag = [0] * len(words)
    
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word in vocab position
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    
    return np.array(bag)

### Example translating sentence into bag of words

In [20]:
p = bow("what is cca", words)
print(p)
print(classes)

Found in bag: what
Found in bag: is
Found in bag: cca
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0]
['cca_application', 'cca_benefits', 'cca_components', 'cca_compulsory', 'cca_creation', 'cca_facilities', 'cca_list', 'cca_points_about', 'cca_points_how', 'cca_quit', 'sfl_modules', 'sfl_register', 'sfl_schedule', 'sfl_what']


In [21]:
'''
Good practice to save trained model into pickle file to reuse to publish
through Flask REST API
'''

'\nGood practice to save trained model into pickle file to reuse to publish\nthrough Flask REST API\n'

In [22]:
# Use pickle to load in pre-trained model
# global graph
# graph = tf.get_default_graph()

# with open(f"katana-assistant-model.pkl", 'rb') as f:
#     model = pickle.load(f)

In [23]:
'''
before publishing model through Flask REST API, always good to run extra test
use model.predict to classify user input & based on calculated
probability return intent (multiple intents can be returned)
'''

'\nbefore publishing model through Flask REST API, always good to run extra test\nuse model.predict to classify user input & based on calculated\nprobability return intent (multiple intents can be returned)\n'

In [24]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from model
    input_data = pd.DataFrame([bow(sentence, words)], dtype = float,
                              index = ['input'])
    # print([input_data])
    
    results = model.predict([input_data])[0]
    # filter predictions below a threshold & provide intent index
    
#     for i, r in enumerate(results):
#       print(i)
#       print(r)
#       print(r > ERROR_THRESHOLD)
#       print()
    
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key = lambda x: x[1], reverse = True)

    
    return_list = []
    for r in results:
        # print(r)
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent & probability
    return return_list

In [31]:
classify_local('can i get smoothie')

Found in bag: can
Found in bag: i
Found in bag: get


[('cca_compulsory', '0.8018518')]

### Publish same function through REST endpoint